In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [ ]:
cases14_23 = pd.read_csv("../cases_data/casesWNV_countyTX_14_23.csv")
cases14_23.head()

In [ ]:
cases_Class = cases14_23.copy()
for i in cases_Class.drop(columns='County').columns:
    cases_Class.loc[cases_Class[str(i)] > 0, str(i)] = 1

In [ ]:
colNames = {'NAME': 'County', 'estimate': 'Population', 'variable': 'Sex,Age'}
oldAgeCol = ['03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
             '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49']

In [ ]:
pop14 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX14.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop15 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX15.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop16 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX16.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop17 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX17.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop18 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX18.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop19 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX19.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop20 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX20.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop21 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX21.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop22 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX22.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)
pop23 = pd.read_csv("../demographic_data/pop_bySexAge_acsGroups_countyTX23.csv").drop(columns=['Unnamed: 0', 'GEOID', 'moe']).rename(columns=colNames)

In [ ]:
def createSexAgeDf(dfPop):
    dfPop['Sex,Age'] = dfPop['Sex,Age'].str.replace('B01001_0', '').replace({"02": "Total Male", "26": "Total Female"})
    dfPop['County'] = dfPop['County'].str.replace(' County, Texas', '')
    sexAgeDf = dfPop.pivot(index='County', columns='Sex,Age', values='Population')
    return sexAgeDf

In [ ]:
sexAge14 = createSexAgeDf(pop14)
sexAge15 = createSexAgeDf(pop15)
sexAge16 = createSexAgeDf(pop16)
sexAge17 = createSexAgeDf(pop17)
sexAge18 = createSexAgeDf(pop18)
sexAge19 = createSexAgeDf(pop19)
sexAge20 = createSexAgeDf(pop20)
sexAge21 = createSexAgeDf(pop21)
sexAge22 = createSexAgeDf(pop22)
sexAge23 = createSexAgeDf(pop23)

In [ ]:
sexAge14.head()

In [ ]:
def createPopGroups4(dfAge, year, caseDf):
    dfNew = dfAge.copy()
    dfNew['Male 0-19'] = dfNew['03'] + dfNew['04'] + dfNew['05'] + dfNew['06'] + dfNew['07'] 
    dfNew['Male 20-59'] = dfNew['08'] + dfNew['09'] + dfNew['10'] + dfNew['11'] + dfNew['12'] + dfNew['13'] + dfNew['14'] + dfNew['15'] + dfNew['16'] + dfNew['17']
    dfNew['Male 60-79'] = dfNew['18'] + dfNew['19'] + dfNew['20'] + dfNew['21'] + dfNew['22'] + dfNew['23']
    dfNew['Male 80+'] = dfNew['24'] + dfNew['25']
    dfNew['Female 0-19'] = dfNew['27'] + dfNew['28'] + dfNew['29'] + dfNew['30'] + dfNew['31']
    dfNew['Female 20-59'] =dfNew['32'] + dfNew['33'] + dfNew['34'] + dfNew['35'] + dfNew['36'] + dfNew['37'] + dfNew['38'] + dfNew['39'] + dfNew['40'] + dfNew['41']
    dfNew['Female 60-79'] = dfNew['42'] + dfNew['43'] + dfNew['44'] + dfNew['45'] + dfNew['46'] + dfNew['47']
    dfNew['Female 80+'] = dfNew['48'] + dfNew['49']
    dfNew.drop(columns=oldAgeCol, inplace=True)
    dfNew['Cases'] = caseDf[str(year)+' Cases'].values
    return dfNew

In [ ]:
sexAge14_class = createPopGroups4(sexAge14, 2014, cases_Class)
sexAge15_class = createPopGroups4(sexAge15, 2015, cases_Class)
sexAge16_class = createPopGroups4(sexAge16, 2016, cases_Class)
sexAge17_class = createPopGroups4(sexAge17, 2017, cases_Class)
sexAge18_class = createPopGroups4(sexAge18, 2018, cases_Class)
sexAge19_class = createPopGroups4(sexAge19, 2019, cases_Class)
sexAge20_class = createPopGroups4(sexAge20, 2020, cases_Class)
sexAge21_class = createPopGroups4(sexAge21, 2021, cases_Class)
sexAge22_class = createPopGroups4(sexAge22, 2022, cases_Class)
sexAge23_class = createPopGroups4(sexAge23, 2023, cases_Class)

In [ ]:
sexAgepop14_21_class = pd.concat([sexAge14_class, sexAge15_class, sexAge16_class, sexAge17_class, sexAge18_class, sexAge19_class, sexAge20_class, sexAge21_class])
sexAgepop14_21_class.head()

In [ ]:
sexAgepop14_21_class.to_csv("../demogr14_21_trainSet_class.csv")

In [ ]:
coor_matrix14_21_class = sexAgepop14_21_class.corr()
print(coor_matrix14_21_class['Cases'].sort_values(ascending=False))

In [ ]:
sexAgepop22_23_class = pd.concat([sexAge22_class, sexAge23_class])
sexAgepop22_23_class.to_csv("../demogr22_23_testSet_class.csv")